In [132]:
def global_alignment(s1, s2):
    n, m = len(s1), len(s2)
    
    nlines = n + 1
    ncols = m + 1
    
    matrix = [ [float('-inf') for _ in range(ncols)] for __ in range(nlines)]
    ancestors = [[0 for _ in range(ncols)] for __ in range(nlines)]
    
    for i in range(ncols):
        matrix[0][i] = -i
        ancestors[0][i] = 'left'
    
    for i in range(nlines):
        matrix[i][0] = -i
        ancestors[i][0] = 'up'
    
    for i in range(1, nlines):
        for j in range(1, ncols):
            mismatch = 1 if s1[i-1] == s2[j-1] else -1
                
            diag = matrix[i-1][j-1] + mismatch
            up = matrix[i-1][j] - 1
            left = matrix[i][j-1] - 1
            best = max(diag, up, left)
            
            matrix[i][j] = best
            
            if best == diag:
                ancestors[i][j] = 'diag'
            elif best == up:
                ancestors[i][j] = 'up'
            else:
                ancestors[i][j] = 'left'
       
    line1 = []
    line2 = [] 
    i, j = n, m
    diag_counter = 0
    while i > 0 or j > 0:
        if ancestors[i][j] == 'diag':
            diag_counter += 1
            line1.append(s1[i-1])
            line2.append(s2[j-1])
            i -= 1
            j -= 1
        elif ancestors[i][j] == 'up':
            line1.append(s1[i-1])
            line2.append('_')
            i -= 1
        else:
            line1.append('_')
            line2.append(s2[j-1])
            j -= 1    
        
    line1 = ''.join(line1[::-1])
    line2 = ''.join(line2[::-1])
    return line1 + '\n' + line2

In [148]:
def affine_gap_alignment(s1, s2, weight_match=1, weight_mismatch=1, open_gap_penalty=10, continue_gap_penalty=1):
    assert open_gap_penalty >= 0
    n = len(s1)
    m = len(s2)
    nlines = n + 1
    ncols = m + 1
    
    upper = np.full([nlines, ncols], -np.inf)  # движение сверху вниз, gap в s2
    lower = np.full([nlines, ncols], -np.inf)  # движение слева направо, gap в s1
    middle = np.full([nlines, ncols], -np.inf)
    
    upper[0][0] = lower[0][0] = middle[0][0] = 0
    
    ancestors = [ [0 for _ in range(ncols)] for __ in range(nlines) ]
    
    for i in range(1, nlines):
        upper[i][0] = - open_gap_penalty - continue_gap_penalty * i

    for j in range(1, ncols):
        lower[0][j] = - open_gap_penalty - continue_gap_penalty * j
        
    for i in range(1, nlines):
        for j in range(1, ncols):
            mismatch = weight_match if s1[i-1] == s2[j-1] else - weight_mismatch
            
            best_upper = max(upper[i-1][j] - continue_gap_penalty, 
                              middle[i-1][j] - open_gap_penalty - continue_gap_penalty)
            
            upper[i][j] = best_upper
            lower[i][j] = max(lower[i][j-1] - continue_gap_penalty,
                              middle[i][j-1] - open_gap_penalty - continue_gap_penalty)
            
            best = max(middle[i-1][j-1] + mismatch,
                               upper[i][j],
                               lower[i][j])
            middle[i][j] = best
            
            if best == middle[i-1][j-1] + mismatch:
                ancestors[i][j] = 'diag'
            elif best == upper[i][j]:
                ancestors[i][j] = 'up'
            else:
                ancestors[i][j] = 'left'
            
    line1 = []
    line2 = []
    i, j = n, m
    while i > 0 or j > 0:
        if ancestors[i][j] == 'diag':
            line1.append(s1[i-1])
            line2.append(s2[j-1])
            i -= 1
            j -= 1
        elif ancestors[i][j] == 'up':
            line1.append(s1[i-1])
            line2.append('_')
            i -= 1
        else:
            line1.append('_')
            line2.append(s2[j-1])
            j -= 1
    line1 = ''.join(line1[::-1])
    line2 = ''.join(line2[::-1])
    
    return line1 + '\n' + line2    
    

s1, s2 = 'abaababcbcbcbaccdcccdaddcd', 'acdaadcbababcabc'
print(affine_gap_alignment(s1, s2)) 
print('*' * 10)
print(affine_gap_alignment(s1, s2, 1, 1, 2, 1))

abaababcbcbcbaccdcccdaddcd
a________cda_adcbababcabc_
**********
abaababcbcbcbaccdcccdaddcd
a__cda_adcbabab______cabc_
